In [1]:
# %load Base User.py
import gc
import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF

data = pd.read_csv('../data/UserMovie_train.txt',sep='\t')

In [2]:
# uid 48613  mid 72541
mid = data['mid'].unique()
uid = data['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data['mid'] = data['mid'].map(lambda x:collect_mid_dencoder[x])
data['uid'] = data['uid'].map(lambda x:collect_uid_dencoder[x])

In [3]:
user_matrix = np.zeros([len(uid),len(mid)])
item_matrix = np.zeros([len(mid),len(uid)])
data.head(5)

,uid,mid,timeStamp,star
0,0,0,2007-08-22,5.0
1,1,1,2013-04-24,4.0
2,2,2,2011-12-20,5.0
3,3,3,2006-02-07,3.0
4,4,4,2008-09-22,4.0


In [4]:
DF(user_matrix,dtype=np.uint16).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48613 entries, 0 to 48612
Columns: 72540 entries, 0 to 72539
dtypes: uint16(72540)
memory usage: 6.6 GB


In [5]:
to_make = np.array(data[['uid','mid','star']].values,dtype=np.uint32)
for i in range(len(to_make)):
    if i%1000000 == 0:
        print(i)
    user_matrix[to_make[i][0],to_make[i][1]] = to_make[i][2]
    item_matrix[to_make[i][1],to_make[i][0]] = to_make[i][2]

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000


In [5]:
user_matrix.shape

(48613, 72540)

In [54]:
split_test = user_matrix[:1000]
split_train = user_matrix[1000:5000]
print(split_train.shape)
print(split_test.shape)

(4000, 72540)
(1000, 72540)


In [55]:
# import random
# new_vector = np.zeros(len(mid)) # readline
# for i in range(200000):
#     kfc = random.randint(1,len(uid))
#     new_vector[kfc] = random.randint(0,5)
    
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    
def calc_sim(vec,user_matrix):
    ans = []
    for i in range(len(user_matrix)):
        if i % 10000 ==0:
            print(i)
        #ans.append(np.linalg.norm(vec-user_matrix[i]))
        ans.append(cos_sim(vec,user_matrix[i]))
    return ans

In [56]:
import random
now = 0
final_acc = []
for new_vec in split_test:
    print(now)
    ans = calc_sim(new_vec,split_train)
    to_recommand = DF(ans).reset_index().rename(columns={0:'sim'})
    k = 3
    topk = list(to_recommand.sort_values(by=['sim'],ascending=False).head(k)['index'].values)
#     print(topk)
#     no_to_use = []
#     for i in range(len(new_vec)):
#         if random.randint(0,100) % 10 == 0:
#             if new_vec[i]!=0:
#                 no_to_use.append(i)
    # 推荐没给他看过的
    watch_set = set()
    for i in topk:
        watch_set = watch_set|set(data[data['uid']==i]['mid'].unique())

    watch_set = set(watch_set)-set(no_to_use)
    print('The New One No Watch Movie Length: ',len(watch_set))
    topN = []
    for i in list(watch_set):
        temp = 0
        for j in topk:
            temp += to_recommand[to_recommand['index']==j]['sim'].values*user_matrix[j][i]
        topN.append(temp)

    topN = DF(topN).rename(columns={0:'score'})
    topN['mid'] = watch_set
    need_to_show = 5
    show_video = topN.sort_values(by=['score'],ascending=False).head(need_to_show)['mid'].values
    true_video = [collect_mid_encoder[i] for i in show_video]
#     print(show_video)
#     print(true_video)
#     print(topN.sort_values(by=['score'],ascending=False).head(need_to_show))
    acc = data[(data['mid'].isin(show_video) & (data['uid']==now))].shape[0]/need_to_show
    print('True: ',acc)
    final_acc.append(acc)
    now += 1

0
[2550, 2464, 2044]
The New One No Watch Movie Length:  1456
True:  0.2
0
[1504, 359, 1625]
The New One No Watch Movie Length:  1113
True:  0.4
0
[3050, 1707, 256]
The New One No Watch Movie Length:  935
True:  0.6
0
[3829, 713, 622]
The New One No Watch Movie Length:  325
True:  0.0
0
[3231, 813, 1869]
The New One No Watch Movie Length:  1514
True:  0.2
0
[1788, 1230, 2733]
The New One No Watch Movie Length:  1141
True:  1.2
0
[96, 1916, 3746]
The New One No Watch Movie Length:  1836
True:  0.6
0
[3821, 813, 1869]
The New One No Watch Movie Length:  1538
True:  0.8
0
[3416, 471, 2877]
The New One No Watch Movie Length:  1292
True:  0.2
0
[253, 1183, 3499]
The New One No Watch Movie Length:  1587
True:  45.6
0
[2826, 3527, 3936]
The New One No Watch Movie Length:  806
True:  0.8
0
[1413, 462, 1374]
The New One No Watch Movie Length:  603
True:  1.4
0
[96, 2877, 248]
The New One No Watch Movie Length:  1699
True:  0.4
0
[1905, 122, 308]
The New One No Watch Movie Length:  948
True:  0.

True:  0.0
0
[3936, 468, 2436]
The New One No Watch Movie Length:  3243
True:  2.8
0
[1714, 1210, 2741]
The New One No Watch Movie Length:  1427
True:  0.0
0
[2369, 96, 30]
The New One No Watch Movie Length:  2453
True:  0.0
0
[2076, 547, 3397]
The New One No Watch Movie Length:  1035
True:  0.4
0
[702, 2108, 3128]
The New One No Watch Movie Length:  2308
True:  0.0
0
[1913, 96, 2154]
The New One No Watch Movie Length:  2516
True:  0.4
0
[993, 253, 2669]
The New One No Watch Movie Length:  1485
True:  0.6
0
[218, 2741, 397]
The New One No Watch Movie Length:  2675
True:  0.0
0
[3691, 52, 201]
The New One No Watch Movie Length:  1172
True:  0.4
0
[52, 96, 160]
The New One No Watch Movie Length:  1745
True:  0.4
0
[2922, 3324, 1374]
The New One No Watch Movie Length:  860
True:  0.8
0
[3231, 2281, 383]
The New One No Watch Movie Length:  712
True:  0.0
0
[2154, 795, 3229]
The New One No Watch Movie Length:  1306
True:  0.0
0
[3467, 359, 813]
The New One No Watch Movie Length:  1984
True:

True:  0.2
0
[3511, 3380, 2403]
The New One No Watch Movie Length:  981
True:  0.0
0
[847, 2233, 1504]
The New One No Watch Movie Length:  1383
True:  0.6
0
[3510, 826, 3251]
The New One No Watch Movie Length:  801
True:  0.0
0
[224, 2504, 1563]
The New One No Watch Movie Length:  556
True:  0.2
0
[1070, 3562, 1504]
The New One No Watch Movie Length:  1134
True:  1.2
0
[2049, 3095, 1810]
The New One No Watch Movie Length:  1143
True:  0.4
0
[1230, 3195, 1654]
The New One No Watch Movie Length:  1042
True:  0.4
0
[2394, 2066, 382]
The New One No Watch Movie Length:  1110
True:  0.8
0
[1484, 1654, 2733]
The New One No Watch Movie Length:  461
True:  0.2
0
[468, 12, 1625]
The New One No Watch Movie Length:  3164
True:  0.6
0
[2066, 2835, 98]
The New One No Watch Movie Length:  661
True:  0.8
0
[3933, 3072, 537]
The New One No Watch Movie Length:  939
True:  0.4
0
[870, 1788, 1230]
The New One No Watch Movie Length:  1055
True:  0.0
0
[1374, 3533, 992]
The New One No Watch Movie Length:  1

True:  0.0
0
[3395, 1087, 1794]
The New One No Watch Movie Length:  3174
True:  0.4
0
[3170, 2394, 3936]
The New One No Watch Movie Length:  1230
True:  0.6
0
[2583, 2859, 3511]
The New One No Watch Movie Length:  2466
True:  0.0
0
[52, 1147, 1097]
The New One No Watch Movie Length:  482
True:  0.4
0
[713, 890, 2683]
The New One No Watch Movie Length:  604
True:  0.0
0
[2012, 1696, 1005]
The New One No Watch Movie Length:  1249
True:  0.0
0
[46, 1630, 2072]
The New One No Watch Movie Length:  1436
True:  0.0
0
[79, 375, 446]
The New One No Watch Movie Length:  502
True:  0.0
0
[704, 3517, 1883]
The New One No Watch Movie Length:  1416
True:  0.0
0
[627, 516, 3936]
The New One No Watch Movie Length:  1392
True:  1.0
0
[375, 1012, 887]
The New One No Watch Movie Length:  1009
True:  0.4
0
[1818, 1844, 1788]
The New One No Watch Movie Length:  1771
True:  0.2
0
[2589, 1869, 1736]
The New One No Watch Movie Length:  899
True:  0.6
0
[1413, 537, 2269]
The New One No Watch Movie Length:  825

True:  0.4
0
[1869, 1812, 627]
The New One No Watch Movie Length:  847
True:  0.8
0
[1413, 2269, 2733]
The New One No Watch Movie Length:  303
True:  0.4
0
[1413, 3057, 2394]
The New One No Watch Movie Length:  3020
True:  0.2
0
[2733, 705, 870]
The New One No Watch Movie Length:  628
True:  0.8
0
[1654, 2785, 1899]
The New One No Watch Movie Length:  759
True:  0.2
0
[1386, 1368, 1763]
The New One No Watch Movie Length:  261
True:  0.2
0
[870, 3529, 2012]
The New One No Watch Movie Length:  953
True:  0.4
0
[757, 890, 2733]
The New One No Watch Movie Length:  1041
True:  0.6
0
[252, 2635, 2826]
The New One No Watch Movie Length:  927
True:  0.6
0
[3183, 1258, 2446]
The New One No Watch Movie Length:  522
True:  0.6
0
[2450, 972, 3771]
The New One No Watch Movie Length:  1769
True:  0.4
0
[1736, 2448, 3678]
The New One No Watch Movie Length:  1139
True:  0.0
0
[1504, 738, 518]
The New One No Watch Movie Length:  564
True:  0.0
0
[3510, 2859, 2826]
The New One No Watch Movie Length:  95

True:  0.0
0
[627, 813, 427]
The New One No Watch Movie Length:  1701
True:  0.6
0
[96, 3087, 2394]
The New One No Watch Movie Length:  2207
True:  2.6
0
[1683, 1839, 2755]
The New One No Watch Movie Length:  1333
True:  0.0
0
[2577, 813, 713]
The New One No Watch Movie Length:  1280
True:  0.2
0
[559, 2638, 992]
The New One No Watch Movie Length:  1478
True:  0.2
0
[1043, 1556, 2248]
The New One No Watch Movie Length:  981
True:  0.4
0
[3603, 3746, 359]
The New One No Watch Movie Length:  1337
True:  1.6
0
[1583, 2262, 292]
The New One No Watch Movie Length:  1363
True:  0.2
0
[605, 1042, 2971]
The New One No Watch Movie Length:  421
True:  0.2
0
[2544, 1687, 1058]
The New One No Watch Movie Length:  2014
True:  0.0
0
[2567, 2062, 96]
The New One No Watch Movie Length:  1984
True:  0.2
0
[2120, 1087, 2692]
The New One No Watch Movie Length:  1135
True:  0.2
0
[971, 3780, 3326]
The New One No Watch Movie Length:  900
True:  0.4
0
[1076, 537, 3467]
The New One No Watch Movie Length:  15

True:  0.0
0
[1847, 2129, 317]
The New One No Watch Movie Length:  1112
True:  0.0
0
[1230, 1788, 1654]
The New One No Watch Movie Length:  1145
True:  0.0
0
[2794, 2557, 281]
The New One No Watch Movie Length:  757
True:  0.0
0
[1374, 2934, 906]
The New One No Watch Movie Length:  1445
True:  0.0
0
[441, 3395, 743]
The New One No Watch Movie Length:  467
True:  0.2
0
[315, 3146, 1413]
The New One No Watch Movie Length:  448
True:  0.2
0
[1788, 3957, 2570]
The New One No Watch Movie Length:  1186
True:  0.6
0
[1505, 537, 1044]
The New One No Watch Movie Length:  1032
True:  0.4
0
[3404, 3523, 1397]
The New One No Watch Movie Length:  926
True:  0.2
0
[74, 2814, 3815]
The New One No Watch Movie Length:  1004
True:  0.0
0
[2863, 2567, 2233]
The New One No Watch Movie Length:  1309
True:  0.4
0
[1789, 233, 3814]
The New One No Watch Movie Length:  313
True:  0.0
0
[2826, 63, 3936]
The New One No Watch Movie Length:  1315
True:  0.6
0
[2567, 3170, 3270]
The New One No Watch Movie Length:  

True:  0.0
0
[98, 941, 3939]
The New One No Watch Movie Length:  830
True:  0.2
0
[507, 2583, 2031]
The New One No Watch Movie Length:  2126
True:  0.8
0
[870, 2394, 1365]
The New One No Watch Movie Length:  1263
True:  0.6
0
[3231, 1869, 2733]
The New One No Watch Movie Length:  785
True:  0.8
0
[3000, 3200, 2307]
The New One No Watch Movie Length:  852
True:  0.0
0
[3229, 2394, 216]
The New One No Watch Movie Length:  1736
True:  0.6
0
[3562, 566, 3642]
The New One No Watch Movie Length:  1658
True:  0.2
0
[2826, 1393, 3936]
The New One No Watch Movie Length:  1469
True:  0.0
0
[1413, 890, 96]
The New One No Watch Movie Length:  1831
True:  0.2
0
[2610, 2435, 1512]
The New One No Watch Movie Length:  555
True:  0.0
0
[2859, 2031, 468]
The New One No Watch Movie Length:  3411
True:  0.8
0
[3477, 2652, 2235]
The New One No Watch Movie Length:  1096
True:  0.0
0
[468, 2859, 93]
The New One No Watch Movie Length:  3314
True:  0.2
0
[375, 3351, 3740]
The New One No Watch Movie Length:  99

True:  0.0
0
[1429, 537, 1413]
The New One No Watch Movie Length:  905
True:  0.0
0
[1803, 872, 1230]
The New One No Watch Movie Length:  859
True:  0.0
0
[2567, 3396, 992]
The New One No Watch Movie Length:  901
True:  0.2
0
[359, 96, 1625]
The New One No Watch Movie Length:  2210
True:  0.2
0
[2028, 3385, 1702]
The New One No Watch Movie Length:  2776
True:  0.2
0
[1869, 1864, 2167]
The New One No Watch Movie Length:  583
True:  0.6
0
[578, 3746, 2269]
The New One No Watch Movie Length:  849
True:  0.2
0
[998, 771, 462]
The New One No Watch Movie Length:  1544
True:  0.4
0
[1505, 3572, 1978]
The New One No Watch Movie Length:  665
True:  0.4
0
[1654, 870, 3170]
The New One No Watch Movie Length:  425
True:  0.4
0
[3269, 2705, 3501]
The New One No Watch Movie Length:  1531
True:  0.0
0
[192, 891, 1716]
The New One No Watch Movie Length:  720
True:  0.0
0
[3360, 308, 1698]
The New One No Watch Movie Length:  1274
True:  0.2
0
[1374, 3408, 240]
The New One No Watch Movie Length:  726
Tr

In [57]:
print(np.mean(final_acc))  # 766

0.7262000000000001


In [ ]:
data[data['uid']==0][['mid','star']]

In [56]:
for i in range(len(user_matrix[0])):
    if user_matrix[0][i]!=0:
        print(i)

0
53
79
80
93
113
126
141
145
152
162
172
176
192
196
225
266
268
278
281
289
307
313
347
354
357
358
365
370
410
430
454
461
500
512
515
537
562
563
596
603
615
616
620
622
629
630
649
654
674
687
688
723
738
753
758
794
818
838
849
864
928
949
961
980
1014
1019
1040
1119
1123
1128
1146
1150
1161
1302
1324
1363
1366
1371
1450
1480
1503
1536
1684
1845
1886
1896
1921
1981
2035
2044
2324
2365
2367
2391
2470
2637
2638
2685
2825
2917
2935
2966
3041
3136
3168
3237
3250
3279
3287
3376
3444
3509
3522
3544
3562
3590
3597
3642
3647
3663
3769
3783
3793
3828
3943
4072
4544
4576
4744
5209
5304
5549
6238
6433
6629
8270
8522
11900
16354


In [58]:
# data.sort_values(by=['uid','mid'])
data[data['uid']==0].nunique()

uid            1
mid          140
timeStamp     63
star           3
dtype: int64

1